In [ ]:
#install package
#!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#import file for training
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
df = pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
#preprocessing
df.drop(['keyword', 'location', 'id'], axis=1, inplace=True)
stop = stopwords.words('english')
np.random.seed(19970901)

def url(text):
    stuff = re.compile(r'https?://\S+|www\.\S+')
    return stuff.sub(r'', text)

def emoji(text):
    stuff = re.compile(
        '['
        u'\U0001F600-\U0001F64F'
        u'\U0001F300-\U0001F5FF'
        u'\U0001F680-\U0001F6FF'
        u'\U0001F1E0-\U0001F1FF'
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return stuff.sub(r'', text)

def html(text):
    stuff = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(stuff, '', text)

def punctuation(text):
    stuff = str.maketrans('', '', string.punctuation)
    return text.translate(stuff)

df['text'] = df['text'].apply(lambda x: url(x))
df['text'] = df['text'].apply(lambda x: emoji(x))
df['text'] = df['text'].apply(lambda x: html(x))
df['text'] = df['text'].apply(lambda x: punctuation(x))
df['text'] = df['text'].apply(lambda x: str.lower(x))
#df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
import torch

In [ ]:
#import models to translate to and from romance languages
from transformers import MarianMTModel, MarianTokenizer
target_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name).to('cuda:0')
en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
en_model = MarianMTModel.from_pretrained(en_model_name).to('cuda:0')

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/799k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

In [ ]:
#create a function to translate from English to French (can substitute any language at runtime)
def translate(texts, model, tokenizer, language="fr"):
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(text) for text in texts]
    encoded = tokenizer.prepare_seq2seq_batch(src_texts,return_tensors="pt").to('cuda:0')
    translated = model.generate(**encoded)
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return translated_texts

In [ ]:
#create function to translate back to English based off of translated text
def back_translate(texts, source_lang="en", target_lang="fr"):
    translated_texts = translate(texts, target_model, target_tokenizer, 
                         language=target_lang)
    back_translated_texts = translate(translated_texts, en_model, en_tokenizer, 
                                      language=source_lang)
    return back_translated_texts

In [ ]:
#back translate text from dataset
print(df.head())
#en_texts = ['This is so cool', 'I hated the food', 'They were very helpful']
#aug_texts = back_translate(en_texts, source_lang="en", target_lang="es")
#print(aug_texts)
translated_text = back_translate(list(df['text']), source_lang="en", target_lang="fr")
#print(df.head())

                                                text  target
0  our deeds are the reason of this earthquake ma...       1
1              forest fire near la ronge sask canada       1
2  all residents asked to shelter in place are be...       1
3  13000 people receive wildfires evacuation orde...       1
4  just got sent this photo from ruby alaska as s...       1


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3221: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


RuntimeError: ignored